In [15]:
#Importing necessary modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
import  tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import Dense
from functools import partial
from tensorflow import keras
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam , Adamax



In [16]:
#defining paths of directory
train_dir1="/kaggle/input/chest-xray-pneumonia/chest_xray/train"
image_size=(224,224)
#loading and preprocessing training data
train_ds1 = tf.keras.utils.image_dataset_from_directory(
    train_dir1,
    validation_split=0.1,#defining a validation split
    subset='training',
    seed=123,
    image_size=image_size,#used to resize images to the required image size
    batch_size=32,#creating images of batch size 32
    shuffle=True)

#defining validation datasets
validation_ds1 = tf.keras.utils.image_dataset_from_directory(
    train_dir1,
    validation_split=0.1,
    subset='validation',
    seed=123,
    image_size=image_size,
    batch_size=32,
    shuffle=True)

Found 5216 files belonging to 2 classes.
Using 4695 files for training.
Found 5216 files belonging to 2 classes.
Using 521 files for validation.


In [17]:

class ConcatLayer(Layer):#creating custom classes for concatenating layers
    def __init__(self, **kwargs):
        super(ConcatLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.concat(inputs, axis=3)
#defining inception module
def inception_module(input_layer,f1,f2_convol1,f2_convol3,f3_convol1,f3_convol5,f4):

    #input_layer:input
    # - f1: number of filters of the 1x1 convolutional layer in the first path
    # - f2_convol1, f2_convol3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
    # - f3_convol1, f3_convol5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
    # - f4: number of filters of the 1x1 convolutional layer in the fourth path
    default_conv2d=partial(keras.layers.Conv2D,kernel_size=(3,3),activation='relu',padding='same')
    #1st_path
    path1=default_conv2d(filters=f1,kernel_size=(1,1))(input_layer)
    #2nd_path
    path2=default_conv2d(filters=f2_convol1,kernel_size=(1,1))(input_layer)
    path2=default_conv2d(filters=f2_convol1)(path2)
    #3rd_path
    path3=default_conv2d(filters=f3_convol1,kernel_size=(1,1))(input_layer)
    path3=default_conv2d(filters=f3_convol5,kernel_size=(5,5))(path3)
    #4th_path
    path4= MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
    path4=default_conv2d(filters=f4,kernel_size=1)(path4)
    #Output layer is a concatenation along the depth layer
    output_layer = ConcatLayer()([path1, path2, path3, path4])
    return output_layer

In [18]:
#defining custom class for local response normalisation
class local_response_Normalisation(Layer):
    def _init_(self, **kwargs):
        super(local_response_Normalisation, self)._init_(**kwargs)

    def call(self, M):
        return tf.nn.local_response_normalization(M)
#defining CNN architecture
def CNN_architecture():
    # input layer
    default_conv2d = partial(keras.layers.Conv2D, kernel_size=3, activation='selu', padding='same', strides=1)
    input_layer = Input(shape=(224, 224,3))

    # convolutional layer1: specifications(filters = 64, kernel_size = (7,7), strides = 2)
    M = default_conv2d(filters=64, kernel_size=7, strides=2, padding='valid')(input_layer)

    # max-pooling layer: specifications(pool size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # applying local_response_normalization

    M=local_response_Normalisation()(M)

    # convolutional layer2: specifications(filters = 64, kernel_size = (1,1), strides = 1)
    M = default_conv2d(filters=64, kernel_size=1)(M)

    # convolutional layer3: specifications(filters = 192, kernel_size = (3,3), strides = 1)
    M = default_conv2d(filters=192)(M)

    # applying local_response_normalization
    M=local_response_Normalisation()(M)

    # 2nd max-pooling layer: specifications(pool size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # 1st Inception block
    M = inception_module(M, f1=64, f2_convol1=96, f2_convol3=128, f3_convol1=16, f3_convol5=32, f4=32)

    # 2nd Inception block
    M = inception_module(M, f1=128, f2_convol1=128, f2_convol3=192, f3_convol1=32, f3_convol5=96, f4=64)

    # max-pooling layer: specifications( pool_size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # 3rd Inception block
    M = inception_module(M, f1=192, f2_convol1=96, f2_convol3=208, f3_convol1=16, f3_convol5=48, f4=64)

    # 4th Inception block
    M = inception_module(M, f1=160, f2_convol1=112, f2_convol3=224, f3_convol1=24, f3_convol5=64, f4=64)

    # 5th Inception block
    M = inception_module(M, f1=128, f2_convol1=128, f2_convol3=256, f3_convol1=24, f3_convol5=64, f4=64)

    # 6th Inception block
    M = inception_module(M, f1=112, f2_convol1=144, f2_convol3=288, f3_convol1=32, f3_convol5=64, f4=64)

    # 7th Inception block
    M = inception_module(M, f1=256, f2_convol1=160, f2_convol3=320, f3_convol1=32, f3_convol5=128, f4=128)

    # max-pooling layer: specifications(pool_size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # 8th Inception block
    M = inception_module(M, f1=256, f2_convol1=160, f2_convol3=320, f3_convol1=32, f3_convol5=128, f4=128)

    # 9th Inception block
    M = inception_module(M, f1=384, f2_convol1=192, f2_convol3=384, f3_convol1=48, f3_convol5=128, f4=128)

    # Global Average pooling layer
    M = GlobalAveragePooling2D(name='GAPL')(M)


    M = Dropout(0.4)(M)

    # output layer
    output = Dense(1, activation='sigmoid')(M)

    # model
    model = Model(input_layer, output, name='CNN_architecture')
    return model

model = CNN_architecture()
# model.summary()
model.summary()
#compiling model with the desired loss function and optimizer
model.compile(optimizer=Adamax(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


Model: "CNN_architecture"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_114 (Conv2D)         (None, 109, 109, 64)         9472      ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_26 (MaxPooli  (None, 54, 54, 64)           0         ['conv2d_114[0][0]']          
 ng2D)                                                                                            
                                                                                                  
 local_response__normalisat  (None, 54, 54, 64)           0         ['max_pooling2d

In [19]:
#Early_Stopping used to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#Fitting Model
history = model.fit(train_ds1,
                        epochs= 20,
                        validation_data = validation_ds1,
                        callbacks = early_stopping)

Epoch 1/20
147/147 [==============================] - 56s 279ms/step - loss: 0.5201 - accuracy: 0.7423 - val_loss: 0.3496 - val_accuracy: 0.8349
Epoch 2/20
147/147 [==============================] - 41s 268ms/step - loss: 0.2969 - accuracy: 0.8690 - val_loss: 0.3539 - val_accuracy: 0.8656
Epoch 3/20
147/147 [==============================] - 41s 269ms/step - loss: 0.2208 - accuracy: 0.9084 - val_loss: 0.1960 - val_accuracy: 0.9232
Epoch 4/20
147/147 [==============================] - 42s 278ms/step - loss: 0.1914 - accuracy: 0.9229 - val_loss: 0.2585 - val_accuracy: 0.9040
Epoch 5/20
147/147 [==============================] - 42s 276ms/step - loss: 0.1616 - accuracy: 0.9387 - val_loss: 0.1410 - val_accuracy: 0.9501
Epoch 6/20
147/147 [==============================] - 41s 270ms/step - loss: 0.1174 - accuracy: 0.9568 - val_loss: 0.1685 - val_accuracy: 0.9271
Epoch 7/20
147/147 [==============================] - 42s 269ms/step - loss: 0.0981 - accuracy: 0.9632 - val_loss: 0.1578 - val_ac

In [20]:
model.save('/content/pneumonia/pneumonia_trained.h5')#saving model

In [28]:
train_dir2="/kaggle/input/dataset2"
train_ds2 = tf.keras.utils.image_dataset_from_directory(
    train_dir2,
    validation_split=0.1,#defining a validation split
    subset='training',
    seed=123,
    image_size=image_size,#used to resize images to the required image size
    batch_size=32,#creating images of batch size 32
    shuffle=True)

#defining validation datasets
validation_ds2 = tf.keras.utils.image_dataset_from_directory(
    train_dir2,
    validation_split=0.1,
    subset='validation',
    seed=123,
    image_size=image_size,
    batch_size=32,
    shuffle=True)

Found 12230 files belonging to 2 classes.
Using 11007 files for training.
Found 12230 files belonging to 2 classes.
Using 1223 files for validation.


In [29]:
import tensorflow as tf
import keras

class LocalResponseNormalisation(tf.keras.layers.Layer):
        def _init_(self, **kwargs):
            super(local_response_Normalisation, self)._init_(**kwargs)

        def call(self, M):
            return tf.nn.local_response_normalization(M)
class ConcatLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ConcatLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.concat(inputs, axis=3)

model = tf.keras.models.load_model("/content/pneumonia/pneumonia_trained.h5",custom_objects={"local_response_Normalisation":LocalResponseNormalisation,"ConcatLayer":ConcatLayer})


In [30]:
#Early_Stopping used to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#Fitting Model
history = model.fit(train_ds2,
                        epochs= 20,
                        validation_data = validation_ds2,
                        callbacks = early_stopping)

Epoch 1/20
344/344 [==============================] - 91s 230ms/step - loss: 0.1505 - accuracy: 0.9426 - val_loss: 0.1218 - val_accuracy: 0.9583
Epoch 2/20
344/344 [==============================] - 71s 205ms/step - loss: 0.1185 - accuracy: 0.9563 - val_loss: 0.1644 - val_accuracy: 0.9387
Epoch 3/20
344/344 [==============================] - 71s 205ms/step - loss: 0.0992 - accuracy: 0.9637 - val_loss: 0.1240 - val_accuracy: 0.9575
Epoch 4/20
344/344 [==============================] - 71s 205ms/step - loss: 0.0927 - accuracy: 0.9647 - val_loss: 0.0886 - val_accuracy: 0.9657
Epoch 5/20
344/344 [==============================] - 72s 208ms/step - loss: 0.0757 - accuracy: 0.9729 - val_loss: 0.0962 - val_accuracy: 0.9689
Epoch 6/20
344/344 [==============================] - 71s 205ms/step - loss: 0.0608 - accuracy: 0.9772 - val_loss: 0.1267 - val_accuracy: 0.9534
Epoch 7/20
344/344 [==============================] - 71s 205ms/step - loss: 0.0508 - accuracy: 0.9797 - val_loss: 0.0804 - val_ac

In [31]:
model.save('/content/pneumonia/pneumonia_fullytrained.h5')